In [1]:
import pandas as pd

In [2]:
from  shared.models.steam2 import *

In [3]:
df = pd.read_csv("test/abel_matutes_STEAM2.csv")

In [4]:
df

,Unnamed: 0,l,b,t,qpc,wet_surf_k,wet_surf_a3,cr_nofn,n_screw,n_cabin,...,co2_fact_me,co2_fact_ae,nox_fact_me,nox_fact_ae,sox_me,sox_ae,co2_me,co2_ae,nox_me,nox_ae
0,0,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,5557.714,0.956272,8664305.0,1490.7986,217584.56,37.23186
1,1,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,5593.471,0.956272,8720049.0,1490.7986,218984.47,37.23186
2,2,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,5522.108,0.956272,8608796.0,1490.7986,216190.60,37.23186
3,3,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,5579.700,0.956272,8698581.0,1490.7986,218445.33,37.23186
4,4,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,5522.108,0.956272,8608796.0,1490.7986,216190.60,37.23186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33483,33483,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,6034.466,0.956272,9407547.0,1490.7986,236249.42,37.23186
33484,33484,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,6034.466,0.956272,9407547.0,1490.7986,236249.42,37.23186
33485,33485,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,6034.466,0.956272,9407547.0,1490.7986,236249.42,37.23186
33486,33486,190.5,26.0,6.3,0.600526,2307.281738,4331.829102,0.654888,2,96,...,517.04,517.04,12.984299,12.912785,6034.466,0.956272,9407547.0,1490.7986,236249.42,37.23186


In [5]:
row = df.iloc[0]

In [6]:
row

Unnamed: 0                               0
l                                    190.5
b                                       26
t                                      6.3
qpc                               0.600526
wet_surf_k                         2307.28
wet_surf_a3                        4331.83
cr_nofn                           0.654888
n_screw                                  2
n_cabin                                 96
n_ref_teu                                0
design_draft                          True
waterline                           183.75
type            Passenger/Ro-Ro Cargo Ship
me_rpm                                 500
ae_rpm                                 514
inst_pow_me                          18006
inst_pow_ae                           3420
design_speed                          21.4
sfoc_me                                166
sfoc_ae                                166
time                            1452962240
latitude                           40.7615
longitude  

In [7]:
knot_speed = row['sog']
v_design = row['design_speed']
inst_pow = row['inst_pow_me']
L = row['l']
b = row['b']
t = row['t']
qpc = row['qpc']
k_area = row['wet_surf_k']
a3_area = row['wet_surf_a3']
cr_nofn = row['cr_nofn']
n_screw = row['n_screw']
design_draft = row['design_draft']
waterline = row['waterline']
v_safety = 0.5,
sea_dens = 1038

In [8]:
# Limit speed
if knot_speed > (v_safety + v_design):
    knot_speed = (v_design + v_safety)
if knot_speed < 0:
    knot_speed = 0

# STEAM2
ms_speed = knot_speed * 0.514
fn = calc_froude_number(ms_speed, waterline)
cb = calc_block_coeff(fn)
cr = estimate_resistance_coef(fn, cb, cr_nofn, n_screw,
                              design_draft)
wet_surf = estimate_wet_surf(k_area, a3_area, cb)
reynolds_n = calc_reynolds_number(ms_speed, L)
fric_r = calc_frictional_resistance(reynolds_n, sea_dens, ms_speed,
                                    wet_surf)

res_r = calc_residual_resistance(cr, sea_dens, b, t, ms_speed)
total_res = calc_total_resistance(fric_r, res_r)
propel_power = calc_propelling_power(total_res, ms_speed)
pow = estimate_main_engine_power(ms_speed, total_res, qpc)
#pow = min(inst_pow, pow)  # Limit to installed power

In [9]:
fric_r

397145.2089485158

In [10]:
res_r

400657.51149230415

In [11]:
total_res

797802.72044082

In [12]:
propel_power

8095965.18227128

In [13]:
fric_r

397145.2089485158

In [14]:
total_res

797802.72044082

In [18]:
propel_power = calc_propelling_power(total_res/(1000), ms_speed)

In [19]:
propel_power/qpc

13481.466482964928